In [ ]:
!pip -q install dash opencv-python Pillow plotly parse ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 85.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/qaz812345/TrackNetV3.git

Cloning into 'TrackNetV3'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 240 (delta 99), reused 87 (delta 87), pack-reused 129 (from 1)
Receiving objects: 100% (240/240), 2.82 MiB | 34.32 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [ ]:
%cd TrackNetV3

/content/TrackNetV3


In [ ]:
# Get model checkpoints

!gdown --fuzzy https://drive.google.com/file/d/1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA/view?usp=sharing
!unzip /content/TrackNetV3/TrackNetV3_ckpts.zip

Downloading...
From (original): https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA
From (redirected): https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA&confirm=t&uuid=eb9f5430-36fb-4b66-89dd-ea1bc88fd529
To: /content/TrackNetV3/TrackNetV3_ckpts.zip
100% 132M/132M [00:00<00:00, 252MB/s]
Archive:  /content/TrackNetV3/TrackNetV3_ckpts.zip
   creating: ckpts/
  inflating: ckpts/InpaintNet_best.pt  
  inflating: ckpts/TrackNet_best.pt  


In [ ]:
# Switch to modified TrackNetV3/utils/general.py

!gdown --fuzzy https://drive.google.com/file/d/1XTql92jtrdBU2ZRvMZcRCMRNNupp8GcL/view?usp=sharing
!mv general.py utils/general.py


Downloading...
From (original): https://drive.google.com/uc?id=1XTql92jtrdBU2ZRvMZcRCMRNNupp8GcL
From (redirected): https://drive.google.com/uc?id=1XTql92jtrdBU2ZRvMZcRCMRNNupp8GcL&confirm=t&uuid=ee990ec5-ae67-458d-adaa-65595834503c
To: /content/TrackNetV3/general.py
100% 21.1k/21.1k [00:00<00:00, 68.9MB/s]


# **draw shuttle tracks**

In [ ]:
# Change --video_file name

!python predict.py --video_file /content/syq_anton_rally_1.mp4 --tracknet_file ckpts/TrackNet_best.pt --inpaintnet_file ckpts/InpaintNet_best.pt --save_dir /content/prediction --output_video

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100% 47/47 [00:59<00:00,  1.26s/it]
100% 47/47 [00:00<00:00, 57.09it/s]
[ok] Wrote video to: /content/prediction/syq_anton_rally_1.mp4
Done.


# **draw frames**

In [ ]:
from pathlib import Path
import cv2
from typing import List
from tqdm.auto import tqdm
import ffmpeg
import numpy as np

def render_full_video_overlay(
    video_frames: List[np.ndarray],
    out_path: str,
    fps: float,
):
    # Get video dimensions from the first frame
    if not video_frames:
        print("Warning: video_frames list is empty. Cannot generate overlay.")
        return
    H, W, _ = video_frames[0].shape

    scale = max(H, W) / 1080.0

    FONT_SCALE   = max(0.8, 0.9 * scale) # text size

    # --- FFMPEG LOGIC ---
    process = (
        ffmpeg
        .input('pipe:', format='rawvideo', pix_fmt='bgr24', s=f'{W}x{H}', r=fps)
        .output(
            out_path,
            vcodec='libx264',
            pix_fmt='yuv420p',
            acodec='aac',
            strict='experimental'
        )
        .overwrite_output()
        .run_async(pipe_stdin=True)
    )

    try:
        for i, frame in tqdm(enumerate(video_frames), desc="Writing video: "):
            overlay_frame = frame.copy()

            # Draw frame number
            cv2.putText(overlay_frame, text=f"Frame {i}", org=(50, 200),
                        fontScale=FONT_SCALE*1.2, color=(255, 0, 0), thickness=4, fontFace=cv2.FONT_HERSHEY_SIMPLEX)

            process.stdin.write(overlay_frame.tobytes())

    finally:
        if process.stdin:
            process.stdin.close()
        process.wait()
        print(f"FFmpeg process finished. Video saved to {out_path}")

In [ ]:
# Change if needed

INPUT_VIDEO_PATH = "/content/prediction/syq_anton_rally_1.mp4"

video_path = Path(INPUT_VIDEO_PATH)
if not video_path.exists():
    raise FileNotFoundError(f"Video not found: {video_path}")
video_name = video_path.stem

cap = cv2.VideoCapture(str(INPUT_VIDEO_PATH))
video_fps = cap.get(cv2.CAP_PROP_FPS)

# Read all frames into memory.
all_frames = []
while True:
    ok, frame = cap.read()
    if not ok:
        break
    all_frames.append(frame)

if 'cap' in locals() and cap.isOpened():
    cap.release()

render_full_video_overlay(
    video_frames=all_frames,
    out_path=f"/content/{video_name}_with_frames.mp4",
    fps=video_fps
)

Writing video: : 0it [00:00, ?it/s]

FFmpeg process finished. Video saved to /content/syq_anton_rally_1_with_frames.mp4
